In [28]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET  # Pour parser les fichiers XML

In [29]:
image = Image.open('train/apple_1.jpg')

# Afficher l'image
image.show()

In [42]:
# Chemin du dossier contenant les images et les fichiers XML
dossier = "train/"

# Liste pour stocker les caractéristiques et les labels
donnees = []

# Parcourir le dossier
for nom_fichier in os.listdir(dossier):
    if nom_fichier.endswith(".jpg"):  # On travaille avec les fichiers PNG
        # Chemin de l'image
        chemin_image = os.path.join(dossier, nom_fichier)
        
        # Charger l'image avec PIL
        image = Image.open(chemin_image)
        image = image.resize((64, 64))  # Redimensionner si nécessaire
        image_array = np.array(image).flatten()  # Convertir en tableau numpy et aplatir
        
        # Chemin du fichier XML correspondant
        nom_xml = nom_fichier.replace(".jpg", ".xml")
        chemin_xml = os.path.join(dossier, nom_xml)
        
        # Parser le fichier XML
        tree = ET.parse(chemin_xml)
        root = tree.getroot()
        
        # Extraire les informations du XML
        largeur = int(root.find("size/width").text)
        hauteur = int(root.find("size/height").text)
        profondeur = int(root.find("size/depth").text)
        
        # Extraire les informations sur les objets
        for obj in root.findall("object"):
            nom_objet = obj.find("name").text
            xmin = int(obj.find("bndbox/xmin").text)
            ymin = int(obj.find("bndbox/ymin").text)
            xmax = int(obj.find("bndbox/xmax").text)
            ymax = int(obj.find("bndbox/ymax").text)
            
            # Ajouter les caractéristiques et les informations XML à la liste
            donnees.append({
                "nom_fichier": nom_fichier,
                "image": image_array,
                "largeur": largeur,
                "hauteur": hauteur,
                "profondeur": profondeur,
                "nom_objet": nom_objet,
                "xmin": xmin,
                "ymin": ymin,
                "xmax": xmax,
                "ymax": ymax
            })

            

In [39]:
image_array


array([255, 255, 255, ..., 255, 255, 255], dtype=uint8)

In [43]:
# Convertir les listes en dataframe
df = pd.DataFrame(donnees)

# Afficher le dataframe
print(df.head())

    nom_fichier                                              image  largeur  \
0   apple_1.jpg  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        0   
1  apple_10.jpg  [255, 255, 255, 255, 255, 255, 255, 255, 255, ...     1500   
2  apple_11.jpg  [85, 87, 86, 87, 90, 89, 89, 92, 92, 90, 94, 9...      652   
3  apple_11.jpg  [85, 87, 86, 87, 90, 89, 89, 92, 92, 90, 94, 9...      652   
4  apple_11.jpg  [85, 87, 86, 87, 90, 89, 89, 92, 92, 90, 94, 9...      652   

   hauteur  profondeur nom_objet  xmin  ymin  xmax  ymax  
0        0           3     apple     8    15   331   349  
1     1500           3     apple    56    99  1413  1419  
2      436           3     apple   213    33   459   258  
3      436           3     apple     1    30   188   280  
4      436           3     apple   116     5   337   220  


In [33]:
df.isna().sum()

nom_fichier      0
largeur          0
hauteur          0
profondeur       0
nom_objet        0
              ... 
pixel_16379    437
pixel_16380    437
pixel_16381    437
pixel_16382    437
pixel_16383    437
Length: 16393, dtype: int64

In [34]:
df.columns

Index(['nom_fichier', 'largeur', 'hauteur', 'profondeur', 'nom_objet', 'xmin',
       'ymin', 'xmax', 'ymax', 'pixel_0',
       ...
       'pixel_16374', 'pixel_16375', 'pixel_16376', 'pixel_16377',
       'pixel_16378', 'pixel_16379', 'pixel_16380', 'pixel_16381',
       'pixel_16382', 'pixel_16383'],
      dtype='object', length=16393)

In [44]:
# Normaliser les pixels (valeurs entre 0 et 1)
df['image'] = df['image'].apply(lambda x: x / 255.0)

# Encoder les noms des objets en valeurs numériques
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['nom_objet'] = encoder.fit_transform(df['nom_objet'])

In [19]:
df

,nom_fichier,image,largeur,hauteur,profondeur,nom_objet,xmin,ymin,xmax,ymax
0,apple_1.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,0,3,0,8,15,331,349
1,apple_10.jpg,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1500,1500,3,0,56,99,1413,1419
2,apple_11.jpg,"[0.3333333333333333, 0.3411764705882353, 0.337...",652,436,3,0,213,33,459,258
3,apple_11.jpg,"[0.3333333333333333, 0.3411764705882353, 0.337...",652,436,3,0,1,30,188,280
4,apple_11.jpg,"[0.3333333333333333, 0.3411764705882353, 0.337...",652,436,3,0,116,5,337,220
...,...,...,...,...,...,...,...,...,...,...
460,orange_76.jpg,"[0.996078431372549, 0.6, 0.2, 1.0, 0.6, 0.2, 0...",1620,1080,3,2,494,1,1452,741
461,orange_76.jpg,"[0.996078431372549, 0.6, 0.2, 1.0, 0.6, 0.2, 0...",1620,1080,3,2,822,643,1620,1066
462,orange_8.jpg,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",600,393,3,2,42,86,282,330
463,orange_8.jpg,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",600,393,3,2,291,81,545,348


In [61]:
dataframe = pd.DataFrame(df['image']) 

In [62]:
dataframe

,image
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,"[0.3333333333333333, 0.3411764705882353, 0.337..."
3,"[0.3333333333333333, 0.3411764705882353, 0.337..."
4,"[0.3333333333333333, 0.3411764705882353, 0.337..."
...,...
460,"[0.996078431372549, 0.6, 0.2, 1.0, 0.6, 0.2, 0..."
461,"[0.996078431372549, 0.6, 0.2, 1.0, 0.6, 0.2, 0..."
462,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
463,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."


In [55]:
!pip install opencv-python

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


In [59]:
train_images = []       
train_labels = []
shape = (200,200)  
train_path = 'train/'

for filename in os.listdir(train_path):
    if filename.split('.')[1] == 'jpg':
        img = cv2.imread(os.path.join(train_path,filename))
        
        # Spliting file names and storing the labels for image in list
        train_labels.append(filename.split('_')[0])
        
        # Resize all images to a specific shape
        img = cv2.resize(img,shape)
        
        train_images.append(img)

# Converting labels into One Hot encoded sparse matrix
train_labels = pd.get_dummies(train_labels).values

# Converting train_images to array
train_images = np.array(train_images)

# Splitting Training data into train and validation dataset
x_train,x_val,y_train,y_val = train_test_split(train_images,train_labels,random_state=1)


In [ ]:
train_labels

In [60]:
train_images

array([[[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        ...,

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
        

In [ ]:
x_train,x_val,y_train,y_val = train_test_split(train_images,train_labels,random_state=1)


In [66]:
from sklearn.ensemble import RandomForestClassifier

# Initialiser le modèle
modele = RandomForestClassifier()

# Entraîner le modèle
#modele.fit(x_train.drop(columns=["nom_fichier", "image"]), y_train)  # On exclut les colonnes non numériques

# Évaluer le modèle
score = modele.score(x_val.drop(columns=["nom_fichier", "image"]), y_val)
print(f"Précision du modèle : {score * 100:.2f}%")

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [22]:
from sklearn.model_selection import train_test_split

# Séparer les caractéristiques (X) et les labels (y)
X = df.drop(columns=["nom_objet"])  # Toutes les colonnes sauf le label
y = df['nom_objet']  # Le label

# Séparer en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
from sklearn.ensemble import RandomForestClassifier

# Initialiser le modèle
modele = RandomForestClassifier()

# Entraîner le modèle
modele.fit(X_train.drop(columns=["nom_fichier", "image"]), y_train)  # On exclut les colonnes non numériques

# Évaluer le modèle
score = modele.score(X_test.drop(columns=["nom_fichier", "image"]), y_test)
print(f"Précision du modèle : {score * 100:.2f}%")

Précision du modèle : 60.22%


descripteurs opencv

In [109]:
import cv2


def extract_color_histogram(image, bins=(8, 8, 8)):
    # Convertir l'image en espace de couleur HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Calculer l'histogramme en 3D pour H, S et V
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    
    # Normaliser l'histogramme
    cv2.normalize(hist, hist)
    
    return hist.flatten()


In [ ]:
def extract_shape_features(image):
    # Convertir en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Seuillage pour obtenir un contour net
    _, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    
    # Trouver les contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Si des contours existent, calculer les moments de Hu
    if contours:
        cnt = max(contours, key=cv2.contourArea)  # Prendre le plus grand contour
        moments = cv2.moments(cnt)
        hu_moments = cv2.HuMoments(moments).flatten()
        
        # Normalisation log
        hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)
        
        return hu_moments
    return None
image = cv2.imread(chemin_image)


In [106]:


def extract_hog_features(image):
    # Convertir en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Vérifier que l'image n'est pas vide
    if gray is None or gray.size == 0:
        print("Erreur : Image vide pour HOG")
        return np.array([])  # Retourne un tableau vide si problème

    # Redimensionner l'image à une taille standard (64x128 pour HOG)
    gray = cv2.resize(gray, (64, 128))

    # Définir le descripteur HOG
    hog = cv2.HOGDescriptor()

    # Calculer les descripteurs HOG
    hog_features = hog.compute(gray)

    # Vérifier si le calcul est réussi
    if hog_features is None:
        print("Erreur : Impossible de calculer HOG")
        return np.array([])

    return hog_features.flatten()


Yolo

In [82]:
from ultralytics import YOLO
import cv2



In [123]:

# Charger le modèle YOLO
model = YOLO("yolov8n.pt")

# Dossiers
dossier_origin = "train/"
output_folder = "detected_objects"

# Vérifier et créer le dossier de sortie
os.makedirs(output_folder, exist_ok=True)

# Parcourir toutes les images dans le dossier d'origine
for k, nom_fichier in enumerate(os.listdir(dossier_origin)):
    if nom_fichier.endswith(".jpg"): 
        image_path = os.path.join(dossier_origin, nom_fichier)
        image = cv2.imread(image_path)  # Charger l'image

        if image is None:
            print(f"Erreur : Impossible de charger {image_path}")
            continue

        # Faire la détection sur cette image
        results = model(image)

        # Extraire les objets détectés
        for i, (box, cls) in enumerate(zip(results[0].boxes.xyxy, results[0].boxes.cls)):
            x_min, y_min, x_max, y_max = map(int, box)  # Convertir en entiers
            class_name = model.names[int(cls)]  # Obtenir le nom de la classe détectée

            # Rogner l'objet détecté
            cropped_object = image[y_min:y_max, x_min:x_max]

            # Vérifier si le crop est valide
            if cropped_object.shape[0] == 0 or cropped_object.shape[1] == 0:
                print(f"Avertissement : Objet {class_name} ignoré (zone trop petite)")
                continue

            # Construire le chemin du fichier avec le nom de l'image d'origine
            if class_name in {"banana", "apple", "orange"}:
                filename = os.path.join(output_folder, f"{class_name}_{k}_{i}.jpg")

                # Sauvegarder l'image découpée
                cv2.imwrite(filename, cropped_object)
                print(f"Objet détecté enregistré : {filename}")

print("Extraction terminée.")



0: 640x640 1 apple, 133.0ms
Speed: 4.4ms preprocess, 133.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Objet détecté enregistré : detected_objects\apple_0_0.jpg

0: 640x640 1 apple, 116.9ms
Speed: 5.3ms preprocess, 116.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
Objet détecté enregistré : detected_objects\apple_2_0.jpg

0: 448x640 3 apples, 1 carrot, 102.1ms
Speed: 2.2ms preprocess, 102.1ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)
Objet détecté enregistré : detected_objects\apple_4_0.jpg
Objet détecté enregistré : detected_objects\apple_4_1.jpg
Objet détecté enregistré : detected_objects\apple_4_2.jpg

0: 640x640 2 apples, 138.6ms
Speed: 3.9ms preprocess, 138.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Objet détecté enregistré : detected_objects\apple_6_0.jpg
Objet détecté enregistré : detected_objects\apple_6_1.jpg

0: 640x640 3 apples, 106.1ms
Speed: 4.8ms preprocess, 106.1ms inference, 

Extraction des descripteurs avec les images yolo

In [124]:

# Dossier contenant les images extraites par YOLO
dossier = "detected_objects"

# Initialisation d'un tableau pour stocker les descripteurs
tab_descripteurs = []

# Parcourir toutes les images du dossier
for nom_fichier in os.listdir(dossier):
    if nom_fichier.endswith(".jpg"): 
        # Charger l'image
        chemin_image = os.path.join(dossier, nom_fichier)
        image = cv2.imread(chemin_image)

        if image is None:
            print(f"Erreur : Impossible de charger {chemin_image}")
            continue  # Passe à l'image suivante

        # Extraire les descripteurs
        color_features = extract_color_histogram(image)
        shape_features = extract_shape_features(image)
        hog_features = extract_hog_features(image)

        # Vérifier que les descripteurs existent
        if shape_features is None or len(shape_features) == 0:
            shape_features = np.array([0])  # Valeur par défaut si aucun contour

        # Concaténer les descripteurs en un seul vecteur
        feature_vector = np.concatenate([color_features, shape_features, hog_features])

        # Ajouter le vecteur au tableau des descripteurs
        tab_descripteurs.append(feature_vector)


# Convertir la liste en un tableau NumPy final
if tab_descripteurs:
    tab_descripteurs = np.vstack(tab_descripteurs)
else:
    tab_descripteurs = np.array([])  # Tableau vide si aucune image n'a été traitée
    print(" Aucun descripteur n'a été extrait.")



In [125]:
tab_descripteurs

array([[          0,           0,           0, ...,           0,           0,  0.00086419],
       [          0,           0,           0, ...,     0.13733,     0.20951,     0.20486],
       [          0,           0,           0, ...,           0,           0,   8.452e-06],
       ...,
       [          0,           0,           0, ...,           0,           0,    0.004643],
       [          0,           0,           0, ...,     0.05536,     0.13809,     0.28905],
       [          0,           0,           0, ...,     0.10506,   0.0072423,   0.0016069]])

test avec un mlp 

In [112]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [126]:
X = tab_descripteurs  # Ton tableau numpy (features)

labels = [nom.split("_")[0] for nom in os.listdir("detected_objects") if nom.endswith(".jpg")]
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
y = encoded_labels  # Une liste ou array numpy des classes associées

# Normaliser les données (important pour MLP)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Séparer en train/test (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=1)

# Définir le modèle MLP
mlp = MLPClassifier(hidden_layer_sizes=(30, 10), activation='relu', max_iter=500)

# Entraîner le modèle
mlp.fit(X_train, y_train)

# Prédire sur le test set
y_pred = mlp.predict(X_test)

# Évaluer la précision
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle : {accuracy:.2f}")


Précision du modèle : 0.76


In [113]:
from sklearn.preprocessing import LabelEncoder

découper les images avec les box de la database 

In [116]:

filename = "apple1.jpg"



apple1.xml


In [119]:

# Dossiers
dossier_origin = "train/"
output_folder = "prechosen_objects"

# Vérifier et créer le dossier de sortie
os.makedirs(output_folder, exist_ok=True)

# Parcourir toutes les images dans le dossier d'origine
for k, nom_fichier in enumerate(os.listdir(dossier_origin)):
    if nom_fichier.endswith(".jpg"): 
        image_path = os.path.join(dossier_origin, nom_fichier)
        image = cv2.imread(image_path)  # Charger l'image

        if image is None:
            print(f"Erreur : Impossible de charger {image_path}")
            continue

        # Chemin du fichier XML correspondant
        nom_xml = nom_fichier.replace(".jpg", ".xml")
        chemin_xml = os.path.join(dossier_origin, nom_xml)
        
        # Parser le fichier XML
        tree = ET.parse(chemin_xml)
        root = tree.getroot()
        
        # Extraire les informations sur les objets
        for obj in root.findall("object"):
            nom_objet = obj.find("name").text
            x_min = int(obj.find("bndbox/xmin").text)
            y_min = int(obj.find("bndbox/ymin").text)
            x_max = int(obj.find("bndbox/xmax").text)
            y_max = int(obj.find("bndbox/ymax").text)
            

        # Extraire les objets détectés
        for i, (box, cls) in enumerate(zip(results[0].boxes.xyxy, results[0].boxes.cls)):

            # Rogner l'objet détecté
            cropped_object = image[y_min:y_max, x_min:x_max]

            # Vérifier si le crop est valide
            if cropped_object.shape[0] == 0 or cropped_object.shape[1] == 0:
                print(f"Avertissement : Objet ignoré (zone trop petite)")
                continue

            # Construire le chemin du fichier avec le nom de l'image d'origine
            filename = os.path.join(output_folder, f"{k}_{i}.jpg")

            # Sauvegarder l'image découpée
            cv2.imwrite(filename, cropped_object)
            print(f"Objet détecté enregistré : {filename}")

print("Extraction terminée.")


Objet détecté enregistré : prechosen_objects\0_0.jpg
Objet détecté enregistré : prechosen_objects\0_1.jpg
Objet détecté enregistré : prechosen_objects\0_2.jpg
Objet détecté enregistré : prechosen_objects\2_0.jpg
Objet détecté enregistré : prechosen_objects\2_1.jpg
Objet détecté enregistré : prechosen_objects\2_2.jpg
Objet détecté enregistré : prechosen_objects\4_0.jpg
Objet détecté enregistré : prechosen_objects\4_1.jpg
Objet détecté enregistré : prechosen_objects\4_2.jpg
Objet détecté enregistré : prechosen_objects\6_0.jpg
Objet détecté enregistré : prechosen_objects\6_1.jpg
Objet détecté enregistré : prechosen_objects\6_2.jpg
Objet détecté enregistré : prechosen_objects\8_0.jpg
Objet détecté enregistré : prechosen_objects\8_1.jpg
Objet détecté enregistré : prechosen_objects\8_2.jpg
Objet détecté enregistré : prechosen_objects\10_0.jpg
Objet détecté enregistré : prechosen_objects\10_1.jpg
Objet détecté enregistré : prechosen_objects\10_2.jpg
Objet détecté enregistré : prechosen_object

In [128]:
# Dossiers
dossier_origin = "train/"
output_folder = "prechosen_objects"

# Vérifier et créer le dossier de sortie
os.makedirs(output_folder, exist_ok=True)
labels = []
# Parcourir toutes les images dans le dossier d'origine
for k, nom_fichier in enumerate(os.listdir(dossier_origin)):
    if nom_fichier.endswith(".jpg"): 
        image_path = os.path.join(dossier_origin, nom_fichier)
        image = cv2.imread(image_path)  # Charger l'image

        if image is None:
            print(f"Erreur : Impossible de charger {image_path}")
            continue

        # Chemin du fichier XML correspondant
        nom_xml = nom_fichier.replace(".jpg", ".xml")
        chemin_xml = os.path.join(dossier_origin, nom_xml)

        # Vérifier si le fichier XML existe
        if not os.path.exists(chemin_xml):
            print(f"Avertissement : Pas de fichier XML pour {nom_fichier}, image ignorée.")
            continue

        # Parser le fichier XML
        tree = ET.parse(chemin_xml)
        root = tree.getroot()

        # Extraire et enregistrer les objets détectés
        for i, obj in enumerate(root.findall("object")):
            nom_objet = obj.find("name").text
            x_min = int(obj.find("bndbox/xmin").text)
            y_min = int(obj.find("bndbox/ymin").text)
            x_max = int(obj.find("bndbox/xmax").text)
            y_max = int(obj.find("bndbox/ymax").text)

            # Rogner l'objet détecté
            cropped_object = image[y_min:y_max, x_min:x_max]

            # Vérifier si le crop est valide
            if cropped_object.shape[0] == 0 or cropped_object.shape[1] == 0:
                print(f"Avertissement : Objet ignoré (zone trop petite)")
                continue

            # Construire le chemin du fichier avec le nom de l'image d'origine
            filename = os.path.join(output_folder, f"{k}_{i}.jpg")
            labels.append(nom_objet)

            # Sauvegarder l'image découpée
            cv2.imwrite(filename, cropped_object)
            print(f"Objet détecté enregistré : {filename}")

print("Extraction terminée.")


Objet détecté enregistré : prechosen_objects\0_0.jpg
Objet détecté enregistré : prechosen_objects\2_0.jpg
Objet détecté enregistré : prechosen_objects\4_0.jpg
Objet détecté enregistré : prechosen_objects\4_1.jpg
Objet détecté enregistré : prechosen_objects\4_2.jpg
Objet détecté enregistré : prechosen_objects\6_0.jpg
Objet détecté enregistré : prechosen_objects\8_0.jpg
Objet détecté enregistré : prechosen_objects\8_1.jpg
Objet détecté enregistré : prechosen_objects\8_2.jpg
Objet détecté enregistré : prechosen_objects\10_0.jpg
Objet détecté enregistré : prechosen_objects\12_0.jpg
Objet détecté enregistré : prechosen_objects\14_0.jpg
Objet détecté enregistré : prechosen_objects\16_0.jpg
Objet détecté enregistré : prechosen_objects\18_0.jpg
Objet détecté enregistré : prechosen_objects\20_0.jpg
Objet détecté enregistré : prechosen_objects\22_0.jpg
Objet détecté enregistré : prechosen_objects\24_0.jpg
Objet détecté enregistré : prechosen_objects\26_0.jpg
Objet détecté enregistré : prechosen_

extraction des features opencv

In [132]:

dossier = "train"

# Initialisation d'un tableau pour stocker les descripteurs
tab_descripteurs = []

# Parcourir toutes les images du dossier
for nom_fichier in os.listdir(dossier):
    if nom_fichier.endswith(".jpg"): 
        # Charger l'image
        chemin_image = os.path.join(dossier, nom_fichier)
        image = cv2.imread(chemin_image)

        if image is None:
            print(f"Erreur : Impossible de charger {chemin_image}")
            continue  # Passe à l'image suivante

        # Extraire les descripteurs
        color_features = extract_color_histogram(image)
        shape_features = extract_shape_features(image)
        hog_features = extract_hog_features(image)

        # Vérifier que les descripteurs existent
        if shape_features is None or len(shape_features) == 0:
            shape_features = np.array([0])  # Valeur par défaut si aucun contour

        # Concaténer les descripteurs en un seul vecteur
        feature_vector = np.concatenate([color_features, shape_features, hog_features])

        # Ajouter le vecteur au tableau des descripteurs
        tab_descripteurs.append(feature_vector)


# Convertir la liste en un tableau NumPy final
if tab_descripteurs:
    tab_descripteurs = np.vstack(tab_descripteurs)
else:
    tab_descripteurs = np.array([])  # Tableau vide si aucune image n'a été traitée
    print(" Aucun descripteur n'a été extrait.")


In [140]:
def get_features(directory):
    dossier = directory

    # Initialisation d'un tableau pour stocker les descripteurs
    tab_descripteurs = []

    # Parcourir toutes les images du dossier
    for nom_fichier in os.listdir(dossier):
        if nom_fichier.endswith(".jpg"): 
            # Charger l'image
            chemin_image = os.path.join(dossier, nom_fichier)
            image = cv2.imread(chemin_image)

            if image is None:
                print(f"Erreur : Impossible de charger {chemin_image}")
                continue  # Passe à l'image suivante

            # Extraire les descripteurs
            color_features = extract_color_histogram(image)
            shape_features = extract_shape_features(image)
            hog_features = extract_hog_features(image)

            # Vérifier que les descripteurs existent
            if shape_features is None or len(shape_features) == 0:
                shape_features = np.array([0])  # Valeur par défaut si aucun contour

            # Concaténer les descripteurs en un seul vecteur
            feature_vector = np.concatenate([color_features, shape_features, hog_features])

            # Ajouter le vecteur au tableau des descripteurs
            tab_descripteurs.append(feature_vector)


    # Convertir la liste en un tableau NumPy final
    if tab_descripteurs:
        tab_descripteurs = np.vstack(tab_descripteurs)
    else:
        tab_descripteurs = np.array([])  # Tableau vide si aucune image n'a été traitée
        print(" Aucun descripteur n'a été extrait.")

    return tab_descripteurs

In [141]:
tab = get_features("train")

In [133]:
tab_descripteurs

array([[          0,           0,           0, ...,    0.027098,  0.00056485,     0.38064],
       [          0,           0,           0, ...,           0,           0,           0],
       [ 0.00041294,           0,   0.0041294, ...,     0.39398,     0.32343,      0.1477],
       ...,
       [          0,           0,           0, ...,    0.049234,     0.11226,     0.29747],
       [          0,           0,           0, ...,           0,           0,           0],
       [          0,           0,           0, ...,           0,           0,   0.0015032]])

In [ ]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
y = encoded_labels  # Une liste ou array numpy des classes associées

In [137]:

from sklearn.metrics import accuracy_score, precision_score, f1_score


In [144]:
def get_labels(directory) :
    labels = [nom.split("_")[0] for nom in os.listdir(directory) if nom.endswith(".jpg")]
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    return encoded_labels

In [138]:
X = tab_descripteurs  # tableau numpy (features)
y = get_labels("train")  # Une liste ou array numpy des classes associées

# Normaliser les données (important pour MLP)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Séparer en train/test (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=1)

# Définir le modèle MLP
mlp = MLPClassifier(hidden_layer_sizes=(30, 10), activation='relu', max_iter=500)

# Entraîner le modèle
mlp.fit(X_train, y_train)

# Prédire sur le test set
y_pred = mlp.predict(X_test)

# Évaluer la précision
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Accuracy du modèle : {accuracy:.2f}")
print(f"Précision du modèle : {precision:.2f}")



Accuracy du modèle : 0.64
Précision du modèle : 0.65
